In [1]:
"""
TODO: Enter any documentation that only people updating the metric should read here.

All columns of the solution and submission dataframes are passed to your metric, except for the Usage column.

Your metric must satisfy the following constraints:
- You must have a function named score. Kaggle's evaluation system will call that function.
- You can add your own arguments to score, but you cannot change the first three (solution, submission, and row_id_column_name).
- All arguments for score must have type annotations.
- score must return a single, finite, non-null float.
"""

import pandas as pd
import pandas.api.types
import numpy as np


class ParticipantVisibleError(Exception):
    # If you want an error message to be shown to participants, you must raise the error as a ParticipantVisibleError
    # All other errors will only be shown to the competition host. This helps prevent unintentional leakage of solution data.
    pass

def get_iou_bbox(ground_truth_box, predicted_box):
  '''
  ground_truth_box: 'x' : top_left_x, 'y' : top_left_y, 'width': width, 'height' : height, 'label' : class
  predicted_box: top_left_x, top_left_y, width, height, class
  '''

  intersection_width = np.min((ground_truth_box['x'] + ground_truth_box['width'], predicted_box['x'] + predicted_box['width'])) - np.max((ground_truth_box['x'], predicted_box['x']))

  intersection_height = np.min((ground_truth_box['y'] + ground_truth_box['height'], predicted_box['y'] + predicted_box['height'])) - np.max((ground_truth_box['y'], predicted_box['y']))

  if intersection_height < 0 or intersection_width < 0:
    return 0

  iou = (intersection_height * intersection_width) / (ground_truth_box['width'] * ground_truth_box['height'] + predicted_box['width'] * predicted_box['height'] - intersection_height * intersection_width)

  return iou

def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    '''
    Metric for object detetction, that is a weighted combination of IoU loss and class prediction accuracy.
    '''

    del solution[row_id_column_name]
    del submission[row_id_column_name]    
    
    ious = []
    class_predictions = []
    iou_threshold = 0.5
    alpha = 0.8
    beta = 0.2
    
    for ind in solution.index:
        
        # predictions from the competitor
        predicted_boxes = eval(submission['bboxes'][ind])
        
        # iterate through ground truth boxes of the specific image
        for ground_truth_box in eval(solution['bboxes'][ind]):
            # find the ious between remaining predicted boxes and the ground truth box
            ground_truth_ious = [get_iou_bbox(ground_truth_box, predicted_box) for predicted_box in predicted_boxes]
            ground_truth_ious.append(0)
            
            # find max value
            max_iou = np.max(ground_truth_ious)
            
            if max_iou >= iou_threshold:
                # if over threshold, add to ious array
                ious.append(max_iou)
                
                predicted_box_ind = np.argmax(ground_truth_ious)
                
                # save the label prediction accuracy
                if ground_truth_box['label'] == predicted_boxes[predicted_box_ind]['label']:
                    class_predictions.append(1)
                else:
                    class_predictions.append(0)
                
                # removed the matched box from predicted boxes
                predicted_boxes.pop(predicted_box_ind)
            else:
                # no matches
                ious.append(0)
                class_predictions.append(0)
                
    return alpha * np.mean(ious) + beta * np.mean(class_predictions)
